# Assignment 3

<font color=blue>Name : Neha Patil <br>
ID : 801081006</font>

## Question 1:

*Dataset 1 : 2009_04_28.txt* <br> After reading the conversations, I feel that Mara and Meg are the top two most influential people. I reached this conclusion because I observed that they are most active in accessing the candidates. Also, Mara introduces topics which later turn into debates.<br>

*Dataset 2 : 2009_05_05.txt* <br> After reading the conversations, I feel that Amy and Jordan are the top two most influential people. I reached this conclusion because I observed that Amy is the one who acts as leader, concludes to stop discussion about previous candidate and introduces new candidates' resumes which become the topic of discussion later. Also, Jordan mentions about main characteristics observed from candidates' resumes which later turn into debates.

## Question 2:

**Note :**
1. The StanfordCoreNLP requires running Java Server which can be done using the following command - <br>java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 150000 
2. If the parsed_text function gives an error 'string indices must be integers', it is because of timeout time in that function is less and increasing will resolve the error. 

*Dataset 1 : 2009_04_28.txt* <br> **Turn Length (TL)** measure results show that **Michael and Amy** are the top two most influential people. The results are not the same as that in Question 1. I observed that eventhough Mara introduces topics which turn into debates, she speaks less and more topic oriented. Similarly, although Mara and Meg are according to me, most active in accessing the candidates, they speak to the point and less as compared to others and hence, the results are different. <br>
**Local Topic Introduction (LTI)** measure results show that **Michael and George** are the top two most influential people. The results are not the same as that in Question 1. As Michael is seems to be a top influencer according to both measures, I think I miss calculated his share in Topic Control. Also, I mainly considered introduction and accessment of new candidates as topic introduction but the LTI measure also takes into account other noun phrases which contribute as topic introductions and hence, the results are different.

*Dataset 2 : 2009_05_05.txt* <br> **Turn Length (TL)** measure results show that **Michael and Amy** are the top two most influential people. The results are not the same as that in Question 1 although Amy is second most influential which is close to my observation. Michael talks a lot as compared to others but maybe I did not observe topic introductions in this conversations while reading. I picked Amy as she introduces candidates' resumes everytime which might also mean that she talk more and this drives the results obtained.
**Local Topic Introduction (LTI)** measure results show that **Meg and Amy** are the top two most influential people. The results are not the same as that in Question 1 although Amy is second most influential which is close to my observation. I might have skipped the topics introduced by Meg as I mostly concentrated on introduction of candidates as topics which is done by Amy and hence Amy is one common person whereas Meg is not.

In [1]:
import pandas as pd
import nltk
import operator
import pprint
from nltk.corpus import names
from nltk import word_tokenize
from pycorenlp import StanfordCoreNLP
from fuzzywuzzy.process import dedupe as fuzzy_dedupe
nlp = StanfordCoreNLP('http://localhost:9000')

### Functions for question 2 -

In [21]:
"""The following function extracts noun phrase subtrees from parsed chunk"""
def extract_np(psent):
  for subtree in psent.subtrees():
    if subtree.label() == 'NP':
      yield ' '.join(word for word, tag in subtree.leaves())

In [3]:
"""This function parses tagged sentences and returns noun phrases"""
def nounPhrase(taggedSent):
    nounPhraseList = []
    grammar = r"""
      NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
      {<NNP>+}                # chunk sequences of proper nouns
      {<NN>+}                 # chunk consecutive nouns
      {<DT>?<JJ>*<NN>}
      """
    #grammar = "NP: {<DT>?<JJ>*<NN>}" 
    cp = nltk.RegexpParser(grammar)
    parsed_sent = cp.parse(taggedSent)
    for npstr in extract_np(parsed_sent):
        nounPhraseList.append(npstr)
    return nounPhraseList

In [4]:
"""This function annotates text using coreference resolver"""
def parsed_text(text):
    nlp = StanfordCoreNLP('http://127.0.0.1:9000')
    output = nlp.annotate(text,properties={'timeout': '150000','annotators':'tokenize, ssplit, pos, depparse, parse, dcoref','outputFormat':'json'})
    return output

### Dataset 1 -

In [5]:
data1File1 = pd.read_csv('2009_04_28.txt', sep='(' , names =["gender","text"])
finalFile1 = data1File1.drop(['text'], axis = 1)
data2File1 = pd.read_csv('2009_04_28.txt', sep=':' , names =["names","time1","time2","text"])
finalFile1['totalwords'] = data2File1['text'].str.split().str.len()
finalFile1['text'] = data2File1['text']
print(finalFile1)
LTIdata1 = finalFile1.drop(['totalwords'], axis = 1)

        gender  totalwords                                               text
0     michael            1                                             Hello.
1         amy            1                                             Hello!
2      george            2                                       good evening
3     michael            1                                                 Hi
4     michael           19    Sarah's name has been going between blue and...
5         amy           14    An email just came in from meg that she won'...
6      george            7               we get to hire someone else tonight!
7         amy            9    that leaves mara and Yuan possibly coming la...
8     michael            6                        Oh, hope she finds the cat.
9         amy            2                                            Hi nick
10    michael            1                                             george
11     george            1                                      

### Turn Length -

In [6]:
nameList1 = list(set(data1File1['gender']))
wordCount1 = {}
nameCount1 = {}
TLCount1 = {}
LTICount1 = {}
CCS1 = {}
for name in nameList1:
    wordCount1[name] = 0
    nameCount1[name] = 0
    LTICount1[name] = 0
    CCS1[name] = 0
for index,rows  in finalFile1.iterrows():
    wordCount1[rows['gender']] = wordCount1[rows['gender']] + rows['totalwords']
    nameCount1[rows['gender']] = nameCount1[rows['gender']] + 1
#print (nameCount1)
#print (wordCount1)
for name in nameList1:
    TLCount1[name] = wordCount1[name]/nameCount1[name]
sorted_TLCount1 = sorted(TLCount1.items(), key=operator.itemgetter(1))
for i in range(len(sorted_TLCount1)-1,-1, -1):
    print('Rank of ' + sorted_TLCount1[i][0] + 'is ' + str(len(sorted_TLCount1)-i) + ' with TL count ' + str(sorted_TLCount1[i][1]))

Rank of michael is 1 with TL count 11.405405405405405
Rank of amy is 2 with TL count 7.318518518518519
Rank of mara is 3 with TL count 7.1875
Rank of george is 4 with TL count 7.070588235294117
Rank of nick is 5 with TL count 7.0285714285714285
Rank of michelle is 6 with TL count 4.24
Rank of meg is 7 with TL count 3.8284313725490198


### Local Topic Introduction -

In [7]:
"""Here we get all the noun phrases in NPListTogether1"""
NPListTogether1 = []
path = ("2009_04_28.txt")
with open(path) as sample_text:
    for lines in sample_text:
        lowers = lines.lower()
        text1 = word_tokenize(lowers)
        tags1 = nltk.pos_tag(text1) 
        currentList = nounPhrase(tags1)
        NPListTogether1.append(currentList)
#print(NPListTogether1)

In [8]:
"""We are extracting only the textual data from the files"""
completeText1 = ''
for index,rows  in LTIdata1.iterrows():
    if rows['text'][-1:] != '.' and rows['text'][-1:] != '?' and rows['text'][-1:] != '!':
        completeText1 = completeText1 + (rows['text']) + '.'
    else:
        completeText1 = completeText1 + (rows['text'])
#print(completeText)
s1 = parsed_text(completeText1)

In [9]:
"""We are getting unique coreferences here"""
uniqueCoref1 = []
for key in s1['corefs']:
    for value in s1['corefs'][key]:
        if value['isRepresentativeMention'] == True:
            uniqueCoref1.append(value['text'])
#print(uniqueCoref)

In [10]:
for item in uniqueCoref1:
    flag = 0;
    if flag == 0:
        for i in range(0,len(NPListTogether1)):
            if flag == 0:
                for element in NPListTogether1[i]:
                    if item == element or element in item:
                        if len(element) > 1:
                            LTICount1[LTIdata1['gender'][i]] = LTICount1[LTIdata1['gender'][i]] + 1
                            flag = 1;
                            break

sorted_LTICount1 = sorted(LTICount1.items(), key=operator.itemgetter(1))
for i in range(len(sorted_LTICount1)-1,-1, -1):
    print('Rank of ' + sorted_LTICount1[i][0] + 'is ' + str(len(sorted_LTICount1)-i) + ' with LTI count ' + str(sorted_LTICount1[i][1]))
    

Rank of michael is 1 with LTI count 206
Rank of george is 2 with LTI count 136
Rank of amy is 3 with LTI count 129
Rank of meg is 4 with LTI count 34
Rank of nick is 5 with LTI count 25
Rank of mara is 6 with LTI count 21
Rank of michelle is 7 with LTI count 1


### Dataset 2 -

In [11]:
data1File2 = pd.read_csv('2009_05_05.txt', sep='(' , names =["gender","text"])
finalFile2 = data1File2.drop(['text'], axis = 1)
data2File2 = pd.read_csv('2009_05_05.txt', sep=':' , names =["names","time1","time2","text"])
finalFile2['totalwords'] = data2File2['text'].str.split().str.len()
finalFile2['text'] = data2File2['text']
print(finalFile2)
LTIdata2 = finalFile2.drop(['totalwords'], axis = 1)

        gender  totalwords                                               text
0         meg            2                                          Hi george
1     michael            1                                             Hello.
2         meg            3                                       How are you?
3         meg            1                                                 Hi
4      george            1                                                 hi
5        nick            1                                                 hi
6      george            7                  i gonna heat up instant ramen brb
7        nick            1                                              jinx!
8     michael           11     Can you get out of it by seeing a punch buggy?
9         meg            3                                        I think so!
10        meg            1                                             Hahaha
11    michael           10       {o^o) - there we go. It has whe

### Turn Length -

In [12]:
nameList2 = list(set(data1File2['gender']))
wordCount2 = {}
nameCount2 = {}
TLCount2 = {}
LTICount2 = {}
CCS2 = {}
for name in nameList2:
    wordCount2[name] = 0
    nameCount2[name] = 0
    LTICount2[name] = 0
    CCS2[name] = 0
for index,rows  in finalFile2.iterrows():
    wordCount2[rows['gender']] = wordCount2[rows['gender']] + rows['totalwords']
    nameCount2[rows['gender']] = nameCount2[rows['gender']] + 1
#print (nameCount2)
#print (wordCount2)
for name in nameList2:
    TLCount2[name] = wordCount2[name]/nameCount2[name]
sorted_TLCount2 = sorted(TLCount2.items(), key=operator.itemgetter(1))
for i in range(len(sorted_TLCount2)-1,-1, -1):
    print('Rank of ' + sorted_TLCount2[i][0] + 'is ' + str(len(sorted_TLCount2)-i) + ' with TL count ' + str(sorted_TLCount2[i][1]))

Rank of michael is 1 with TL count 9.0
Rank of amy is 2 with TL count 6.7251461988304095
Rank of george is 3 with TL count 6.285714285714286
Rank of nick is 4 with TL count 5.852941176470588
Rank of mara is 5 with TL count 5.2
Rank of jordan is 6 with TL count 4.7846153846153845
Rank of michelle is 7 with TL count 4.22680412371134
Rank of meg is 8 with TL count 3.757085020242915
Rank of john is 9 with TL count 2.506666666666667


### Local Topic Introduction -

In [13]:
"""Here we get all the noun phrases in NPListTogether2"""
NPListTogether2 = []
path = ("2009_05_05.txt")
with open(path) as sample_text:
    for lines in sample_text:
        lowers = lines.lower()
        text1 = word_tokenize(lowers)
        tags1 = nltk.pos_tag(text1) 
        currentList = nounPhrase(tags1)
        NPListTogether2.append(currentList)
#print(NPListTogether2)

In [14]:
"""We are extracting only the textual data from the files"""
completeText2 = ''
for index,rows  in LTIdata2.iterrows():
    if rows['text'][-1:] != '.' and rows['text'][-1:] != '?' and rows['text'][-1:] != '!':
        completeText2 = completeText2 + (rows['text']) + '.'
    else:
        completeText2 = completeText2 + (rows['text'])
#print(completeText)
s2 = parsed_text(completeText2)

In [15]:
"""We are getting unique coreferences here"""
uniqueCoref2 = []
for key in s2['corefs']:
    for value in s2['corefs'][key]:
        if value['isRepresentativeMention'] == True:
            uniqueCoref2.append(value['text'])
#print(uniqueCoref)

In [16]:
for item in uniqueCoref2:
    flag = 0;
    if flag == 0:
        for i in range(0,len(NPListTogether2)):
            if flag == 0:
                for element in NPListTogether2[i]:
                    if item == element or element in item:
                        if len(element) > 1:
                            LTICount2[LTIdata2['gender'][i]] = LTICount2[LTIdata2['gender'][i]] + 1
                            flag = 1;
                            break

sorted_LTICount2 = sorted(LTICount2.items(), key=operator.itemgetter(1))
for i in range(len(sorted_LTICount2)-1,-1, -1):
    print('Rank of ' + sorted_LTICount2[i][0] + 'is ' + str(len(sorted_LTICount2)-i) + ' with LTI count ' + str(sorted_LTICount2[i][1]))
    

Rank of meg is 1 with LTI count 151
Rank of amy is 2 with LTI count 136
Rank of michael is 3 with LTI count 117
Rank of george is 4 with LTI count 46
Rank of nick is 5 with LTI count 43
Rank of michelle is 6 with LTI count 32
Rank of jordan is 7 with LTI count 31
Rank of john is 8 with LTI count 8
Rank of mara is 9 with LTI count 3


## Question 3:

Candidate Citation Score (CCS) measure counts the number of times each participant talks about the candidates under review. I believe this measure will be helpful because the outcome of both the discussions are choosing a candidate the participants find to be worthy for the specified job and this measure takes into account how each participant contributes in the final outcome.


*Dataset 1 : 2009_04_28.txt* <br> **Candidate Citation Score (CCS)** measure results show that **George and Amy** are the top two most influential people. The results are not the same as that in Question 1. I considered Mara and Meg to be the top ones because I thought they are the most active ones in accessing the candidates. Maybe rereading the conversations might change my results. There can be two possibilities either I miss calculated in determining the top influencers or CCS measure should be combined with other measure to better find the top influencers.

*Dataset 2 : 2009_05_05.txt* <br> **Candidate Citation Score (CCS)** measure results show that **Michelle and George** are the top two most influential people. The results are not the same as that in Question 1. I considered Amy and Jordan to be the top ones because I considered only introduction of candidates and their characteristics as main topics. Maybe rereading the conversations might change my results. This idea of this measure is completely different than the one I used in Question 1 and hence the results are different.

### Functions for question 3 - 

In [17]:
"""This function returns the set of people found within the sentence"""
def proc_sentence(tokens):
    people = set()
    token_count = 0
    for i in range(len(tokens)):
        if token_count < len(tokens):
            person = ''
            token = tokens[token_count]
            if token['ner'] == 'PERSON':
                person += token['word'].lower()
                checking = True
                while checking == True:
                    if token_count + 1 < len(tokens):
                        if tokens[token_count + 1]['ner'] == 'PERSON':
                            token_count += 1
                            person += ' {}'.format(tokens[token_count]['word'].lower())
                        else:
                            checking = False
                            token_count += 1
                    else:
                        checking = False
                        token_count += 1
            else:
                token_count += 1
            if person != '':
                people.add(person)
    return people

In [18]:
"""Using fuzzywuzzy to take care of partial representations of entities"""
def proc_article(article):
    output = nlp.annotate(article, properties={
      'annotators': 'ner',
      'outputFormat': 'json'
      })
    
    people_super = set()
    for sent in output['sentences']:
        people = proc_sentence(sent['tokens'])
        for person in people:
            people_super.add(person)

    contains_dupes = list(people_super)
    
    deduped = fuzzy_dedupe(contains_dupes)
    
    return deduped

### Candidate Citation Score - 

### Dataset 1 -

In [19]:
allNames1 =  list(proc_article(completeText1))
for index,rows  in finalFile1.iterrows():
    currentText = rows['text']
    for element in allNames1:
        if element in currentText:
            CCS1[rows['gender']] = CCS1[rows['gender']] + 1;
sorted_CCS1 = sorted(CCS1.items(), key=operator.itemgetter(1))
for i in range(len(sorted_CCS1)-1,-1, -1):
    print('Rank of ' + sorted_CCS1[i][0] + 'is ' + str(len(sorted_CCS1)-i) + ' with Candidate Citation score ' + str(sorted_CCS1[i][1]))
    

Rank of george is 1 with Candidate Citation count 30
Rank of amy is 2 with Candidate Citation count 19
Rank of mara is 3 with Candidate Citation count 11
Rank of meg is 4 with Candidate Citation count 7
Rank of nick is 5 with Candidate Citation count 1
Rank of michelle is 6 with Candidate Citation count 1
Rank of michael is 7 with Candidate Citation count 0


### Dataset 2 -

In [20]:
allNames2 =  list(proc_article(completeText2))
for index,rows  in finalFile2.iterrows():
    currentText = rows['text']
    for element in allNames2:
        if element in currentText:
            CCS2[rows['gender']] = CCS2[rows['gender']] + 1;
sorted_CCS2 = sorted(CCS2.items(), key=operator.itemgetter(1))
for i in range(len(sorted_CCS2)-1,-1, -1):
    print('Rank of ' + sorted_CCS2[i][0] + 'is ' + str(len(sorted_CCS2)-i) + ' with Candidate Citation score ' + str(sorted_CCS2[i][1]))
    

Rank of michelle is 1 with Candidate Citation count 11
Rank of george is 2 with Candidate Citation count 10
Rank of amy is 3 with Candidate Citation count 9
Rank of john is 4 with Candidate Citation count 2
Rank of michael is 5 with Candidate Citation count 2
Rank of meg is 6 with Candidate Citation count 1
Rank of jordan is 7 with Candidate Citation count 1
Rank of nick is 8 with Candidate Citation count 0
Rank of mara is 9 with Candidate Citation count 0


## Question 4:

Modeling influence is an interesting topic. I learned new concepts like co-reference resolver and network centrality. Along the way, I learned about different libraries and their uses like, fuzzywuzzy and networkx. Most important outcome from this assignment was learning about the StanfordCoreNLP framework, the tools it provides and understanding how to apply the tools to get desired results. <br>
Also, coming up with relevant measures for modeling influence is an important task. I also encountered that Local Topic Introduction measure can also be derived using LDA. 

## References :
1. http://brandonrose.org/ner2sna
2. https://stackoverflow.com/questions/33815401/nltk-how-do-i-traverse-a-noun-phrase-to-return-list-of-strings
3. https://www.analyticsvidhya.com/blog/2019/02/stanfordnlp-nlp-library-python/